# Gaussian Process baseline

This notebook runs a standard Gaussian Process in GPyTorch on ESOL dataset using physiochemical descriptors to provide a baseline.

*Prepared by Maxim Ziatdinov*

Install GPyTorch for GP and DeepChem for data processing:

In [ ]:
!pip install gpytorch
!pip install deepchem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Download dataset:

In [ ]:
!wget https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv

--2025-02-23 05:54:14--  https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96699 (94K) [text/plain]
Saving to: ‘delaney-processed.csv’

delaney-processed.c 100%[===================>]  94.43K  --.-KB/s    in 0.009s  

2025-02-23 05:54:14 (10.4 MB/s) - ‘delaney-processed.csv’ saved [96699/96699]



Import neccessary libraries:

In [ ]:
import gpytorch

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm


import torch

from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen

Load and process data:

In [ ]:
esol_df = pd.read_csv('delaney-processed.csv')
esol_df

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.770,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.300,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.060,CC(C)=CCCC(C)=CC(=O)
3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.870,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43
4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.330,c1ccsc1
...,...,...,...,...,...,...,...,...,...,...
1123,halothane,-2.608,1,197.381,0,0,0,0.00,-1.710,FC(F)(F)C(Cl)Br
1124,Oxamyl,-0.908,1,219.266,1,0,1,71.00,0.106,CNC(=O)ON=C(SC)C(=O)N(C)C
1125,Thiometon,-3.323,1,246.359,0,0,7,18.46,-3.091,CCSCCSP(=S)(OC)OC
1126,2-Methylbutane,-2.245,1,72.151,0,0,1,0.00,-3.180,CCC(C)C


In [ ]:
def calculate_focused_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return {
            'MolWt': Descriptors.ExactMolWt(mol),
            'LogP': Crippen.MolLogP(mol),
            'TPSA': Descriptors.TPSA(mol),
            'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
            'NumHDonors': Descriptors.NumHDonors(mol),
            'NumHAcceptors': Descriptors.NumHAcceptors(mol),
            'NumAromaticRings': Descriptors.NumAromaticRings(mol),
            'FractionCSP3': Descriptors.FractionCSP3(mol),
            'HallKierAlpha': Descriptors.HallKierAlpha(mol),
        }
    return None


# Calculate descriptors
descriptors_list = esol_df['smiles'].apply(calculate_focused_descriptors)
descriptors_df = pd.DataFrame(descriptors_list.tolist())

# Combine with original dataframe
esol_with_descriptors = pd.concat([esol_df, descriptors_df], axis=1)

# Remove any rows with NaN values
esol_with_descriptors = esol_with_descriptors.dropna(subset='measured log solubility in mols per litre')

# Prepare features (X) and target (y)
X = esol_with_descriptors[descriptors_df.columns]
y = esol_with_descriptors['measured log solubility in mols per litre']

X = X.values
y = y.values

Scale inputs:

In [ ]:
x_scaler = StandardScaler()
X = x_scaler.fit_transform(X)

Create training and test datasets:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=1
)

Define and run GP:

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        # Get input dimensionality for ARD
        input_dim = train_x.shape[1]

        self.mean_module = gpytorch.means.ConstantMean()
        # Use ARD=True to learn a separate lengthscale for each input dimension
        self.covar_module = gpytorch.kernels.MaternKernel(ard_num_dims=input_dim)

        # Initialize lengthscales to 1.0
        self.covar_module.lengthscale = torch.ones(input_dim)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_gp(X_train, y_train, n_iterations=1000):
    # Convert numpy arrays to torch tensors if needed
    if isinstance(X_train, np.ndarray):
        X_train = torch.from_numpy(X_train).float()
    if isinstance(y_train, np.ndarray):
        y_train = torch.from_numpy(y_train).float()

    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(X_train, y_train, likelihood)

    # Use the adam optimizer
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},
    ], lr=0.01)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Train the model
    model.train()
    likelihood.train()

    for i in range(n_iterations):
        optimizer.zero_grad()
        output = model(X_train)
        loss = -mll(output, y_train)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Iteration {i+1}/{n_iterations} - Loss: {loss.item():.3f}')

    return model, likelihood

def predict(model, likelihood, X_test):
    # Convert to torch tensor if needed
    if isinstance(X_test, np.ndarray):
        X_test = torch.from_numpy(X_test).float()

    # Get into evaluation (predictive posterior) mode
    model.eval()
    likelihood.eval()

    # Make predictions
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        observed_pred = likelihood(model(X_test))

    # Extract mean and variance
    predictions = observed_pred.mean.numpy()
    variance = observed_pred.variance.numpy()

    return predictions, variance

def compute_nlpd(y_true, y_pred, variance):
    """Compute Negative Log Predictive Density"""
    return -np.mean(-0.5 * np.log(2 * np.pi * variance) -
                    0.5 * (y_true - y_pred)**2 / variance)

def compute_coverage(y_true, y_pred, variance, confidence=0.95):
    """Compute coverage probability"""
    z_score = np.abs(norm.ppf((1 - confidence) / 2))
    lower = y_pred - z_score * np.sqrt(variance)
    upper = y_pred + z_score * np.sqrt(variance)
    return np.mean((y_true >= lower) & (y_true <= upper))

# Train the model
model, likelihood = train_gp(X_train, y_train, n_iterations=1000)

# Make predictions
predictions, variance = predict(model, likelihood, X_test)

# Compute metrics
test_rmse = np.sqrt(np.mean((y_test - predictions) ** 2))
test_r2 = r2_score(y_test, predictions)
test_nlpd = compute_nlpd(y_test, predictions, variance)
test_coverage = compute_coverage(y_test, predictions, variance)

print("Model Performance:")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"NLPD: {test_nlpd:.4f}")
print(f"Coverage probability: {test_coverage:.4f}")

Iteration 100/1000 - Loss: 1.455
Iteration 200/1000 - Loss: 1.319
Iteration 300/1000 - Loss: 1.262
Iteration 400/1000 - Loss: 1.231
Iteration 500/1000 - Loss: 1.213
Iteration 600/1000 - Loss: 1.202
Iteration 700/1000 - Loss: 1.196
Iteration 800/1000 - Loss: 1.192
Iteration 900/1000 - Loss: 1.190
Iteration 1000/1000 - Loss: 1.189
Model Performance:
Test RMSE: 0.6887
Test R²: 0.8876
NLPD: 0.9575
Coverage probability: 0.9291
